In [ ]:
!pip install pandas_schema
!pip install python-louvain
!pip install python-highcharts
!pip install bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-highcharts: filename=python_highcharts-0.4.2-py3-none-any.whl size=60717 sha256=28b4c6d3f9cb83990c807dd1d5bba280e12db1dfe6399602eb6be943f6f2c1f5
  Stored in directory: /root/.cache/pip/wheels/a3/29/b5/6447651b55dd30315e1befe33c548bccfa440eb364f6b74251
Successfully built python-highcharts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import datetime
import numpy as np
import pandas as pd
import csv
import random
from google.colab import files
import pandas as pd
import pandas_schema
from pandas_schema import Column
from pandas_schema.validation import CustomElementValidation
import numpy as np
from decimal import *
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
import networkx as nx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8, Plasma256, d3, Turbo256
from bokeh.transform import linear_cmap
from networkx.algorithms import community
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
output_notebook()
default_date_format ='%d/%m/%Y'
#
def DownloadDataFrame(df, filename):
  df.to_csv(filename, encoding = 'utf-8-sig', index=False, quoting=csv.QUOTE_NONNUMERIC)
  files.download(filename)
#
def ResetDFIndex(df):
  df.index = pd.RangeIndex(len(df.index))
  df.index = range(len(df.index))
  return df
#
def CreateSubDataSet(df, columns, unique, sort):
  df = CreateDatasetFromColabUsers(df)[columns].copy()
  if(sort):
    df = df.sort_values(sort, ascending=True)
    if(unique):
      idx = df[sort].drop_duplicates().index
      event_types = df.loc[idx,:]
  return ResetDFIndex(event_types)
#
def ExtractEventTypes(df, download):
  event_type_ids = CreateSubDataSet(df, ['event_type_id', 'event_type_name'], True, 'event_type_id')
  print(event_type_ids.shape)
  event_type_names = CreateSubDataSet(df, ['event_type_id', 'event_type_name'], True, 'event_type_name')
  print(event_type_names.shape)
  if(len(event_type_ids) != len(event_type_names)):
    print("event_type_id and event_type_name counts are different.", len(event_type_ids), "ids", len(event_type_names), "names")
  if(download):
    DownloadDataFrame(event_type_ids, "event_type_ids.csv")
    DownloadDataFrame(event_type_names, "event_type_names.csv")
  return event_type_ids if len(event_type_ids) < len(event_type_names) else event_type_names
#
def check_decimal(dec):
    try:
        Decimal(dec)
    except InvalidOperation:
        return False
    return True
#
def check_int(num):
    try:
        int(num)
    except ValueError:
        return False
    return True
#
def check_string(test_string):
    return check_null(test_string) and len(str(test_string)) > 0
#
def check_date(date):
  try:
      datetime.datetime.strptime(date, default_date_format)
      return True
  except ValueError:
      return False
  #return check_string(date) and re.match(r'(\d{4})-(\d{2})-(\d{2}) (\d{1,2}):(\d{2})', date) is not None
#
print("check_date should be false: ", check_date(""))
print("check_date should be false: ", check_date("2022-05-18"))
print("check_date should be true: ", check_date("03/03/2013"))
def check_null(d):
    return not(d is None)
#
# define validation elements
decimal_validation = [CustomElementValidation(lambda d: check_decimal(d), 'invalid decimal')]
int_validation = [CustomElementValidation(lambda i: check_int(i), 'invalid integer')]
null_validation = [CustomElementValidation(lambda d: check_null(d), 'field cannot be null')]
string_validation = [CustomElementValidation(lambda s: check_string(s), 'invalid string')]
date_validation = [CustomElementValidation(lambda s: check_date(s), 'invalid date')]
#
#
def validate_colab_events(file, parse_dates):
    print("validating colab_events file")
    # read the data
    data = ReadCSV(file, parse_dates)
    # define validation schema
    schema = pandas_schema.Schema([
            Column('event_id', null_validation+int_validation),
            Column('user_id', null_validation+int_validation),
            Column('description', string_validation),
            Column('status', null_validation + string_validation),
            Column('created_at', date_validation),
            Column('event_type_id', null_validation + int_validation),
            Column('event_type_name', null_validation + string_validation)
            ])
    # apply validation
    errors = schema.validate(data)
    errors_index_rows = [e.row for e in errors]
    data_clean = data.drop(index=errors_index_rows)
    if(len(errors) > 0):
      # save data
      #data_clean.update('"' + data_clean[['description', 'status', 'created_at', 'event_type_name']].astype(str) + '"')
      #data_clean['event_type_id'] = data_clean['event_type_id'].astype('int')
      print(data.shape)
      print(data_clean.shape)
      #DownloadDataFrame(data_clean, "colab_events_cleaned.csv")
      DownloadDataFrame(pd.DataFrame({'errors':errors}), "data_errors.txt")
      print("Errors found on CSV schema. Check out the data_errors.txt")
      return False
    else:
      print("File is clean")
      return True
#
def tipify_colab_events(df):
  df.fillna(-1, inplace=True)
  df['event_id'] = df['event_id'].astype('int')
  df['user_id'] = df['user_id'].astype('int')
  df['event_type_id'] = df['event_type_id'].astype('int')
  return df
#
def validate_colab_users(file, parse_dates):
    # read the data
    print("validating colab_users file")
    data = ReadCSV(file, parse_dates).head(1)
    # define validation schema
    schema = pandas_schema.Schema([
            Column('colab_user_id', null_validation+int_validation),
            Column('gender', null_validation),
            Column('birth_date', null_validation),
            Column('city_id', null_validation),
            Column('city_name', string_validation),
            Column('state_id', null_validation),
            Column('state_name', null_validation + string_validation),
            Column('created_at', date_validation),
            Column('last_sign_in_at', date_validation),
            Column('device', {})
            ])
    print(data.columns)
    # apply validation
    errors = schema.validate(data)
    errors_index_rows = [e.row for e in errors]
    data_clean = data.drop(index=errors_index_rows)
    if(len(errors) > 0):
      # save data
      print(data.shape)
      print(data_clean.shape)
      #DownloadDataFrame(data_clean, "colab_users_cleaned.csv")
      DownloadDataFrame(pd.DataFrame({'errors':errors}), "user_errors.txt")
      print("Errors found on CSV schema. Check out the user_errors.txt")
      return False
    else:
      print("File is clean")
      return True
#
def tipify_colab_users(df):
  df.fillna("", inplace=True)
  df['colab_user_id'] = df['colab_user_id'].astype('int')
  df['city_id'] = df['city_id'].astype('int')
  df['state_id'] = df['state_id'].astype('int')
  return df
#
def tipify_colab_followers(df):
  df.fillna(-1, inplace=True)
  df['source'] = df['source'].astype('int')
  df['target'] = df['target'].astype('int')
  return df
#
def CreateDatasetFromColabEvents(df):
  dataset = df.copy()
  return dataset
#
#
def CreateDatasetFromColabUsers(df):
  dataset = df.copy()
  return dataset
#
def ReadCSV(filename, parse_dates):
  df = pd.read_csv(filename,
                           parse_dates=parse_dates,
                           low_memory=False,
                           quoting=csv.QUOTE_NONNUMERIC,
                           quotechar='"')
  if(parse_dates):
    for field in parse_dates:
      try:
        df[field] = pd.to_datetime(df[field])
        df[field] = df[field].dt.strftime(default_date_format)
      except ValueError:
        print("error parsing date", df[field])
  #df[parse_dates].fillna(0, inplace=True)
  #df[parse_dates] = pd.to_datetime(df[parse_dates])
  #df[parse_dates] = df[parse_dates].dt.strftime('%d/%m/%Y')
  return df
#
def ReadAndValidateColabUsers(file):
  parse_dates = ['created_at', 'last_sign_in_at']
  if (validate_colab_users(file, parse_dates)):
    colab_users = tipify_colab_users(ReadCSV(file, parse_dates))
    return colab_users
#
def ReadAndValidateColabEvents(file):
  parse_dates = ['created_at']
  if (validate_colab_events(file, parse_dates)):
    colab_events = tipify_colab_events(ReadCSV(file, parse_dates))
    return colab_events
#

check_date should be false:  False
check_date should be false:  False
check_date should be true:  True


In [ ]:
import pandas as pd
from geopy.distance import geodesic

colab_events_url_geo = "https://drive.google.com/uc?id=1u-IhA8TtOh5ilRXurEV7TXYN_VIjrWRg&export=download" #All data - with GEO
colab_events_geo = ReadCSV(colab_events_url_geo, None)

# Read in the csv files
users_df = pd.read_csv('colab_users_and_followers.csv')
events_df = colab_events_geo

# Create a dictionary to hold each user's possible home location
home_locations = {}

# Iterate through each user in the users dataframe
for i, user in users_df.iterrows():
    # Get all events associated with the user
    user_events = events_df[events_df['user_id'] == user['colab_user_id']]

    # If the user has no events associated with them, skip to the next user
    if user_events.empty:
        continue

    # Calculate the average lat-lng of all events associated with the user
    total_lat = user_events['lat'].sum()
    total_lng = user_events['lng'].sum()
    avg_lat = total_lat / len(user_events)
    avg_lng = total_lng / len(user_events)

    # Store the user's possible home location in the dictionary
    home_locations[user['colab_user_id']] = (avg_lat, avg_lng)

# Iterate through each user in the users dataframe again, this time adding their home location to the dataframe
for i, user in users_df.iterrows():
    # If the user has a possible home location stored in the dictionary, add it to the dataframe
    if user['colab_user_id'] in home_locations:
        users_df.at[i, 'lat'] = home_locations[user['colab_user_id']][0]
        users_df.at[i, 'lng'] = home_locations[user['colab_user_id']][1]

# Save the updated dataframe to a new CSV file
users_df.to_csv('colab_users_with_home_location.csv', index=False)


In [ ]:
#
# usuários que tem postagens no colab
#
#colab_users_url = "https://drive.google.com/uc?id=1-P0Y9q27tDWQq2HD5PYgySmu1UBlJwO7&export=download" #2020-2022 Niteroi - Clean
#colab_users_url = "https://drive.google.com/uc?id=16gqijd_rNssCV2bWy09cyPJWh4c0ycbe&export=download" #New File (all data)
colab_users_url = "https://drive.google.com/uc?id=1MCyRDc_lty7m2UVKeAFse2vDust2Mtg4&export=download" #05-20 File (all data)
colab_users = ReadAndValidateColabUsers(colab_users_url)
print(colab_users.shape)
colab_users.head(1)
#
# seguidores de usuarios que tem postagens no colab
#
colab_users_followers_url = "https://drive.google.com/uc?id=125MHZFOmhok7avkMpxZiHEejZt9kLkXQ&export=download" #New File (all data)
#colab_users_followers_url = "https://drive.google.com/uc?id=1lbIpMG_AoQrx_dlkDTqTHl9F7EvLlxyC&export=download" #05-20 File (all data)
colab_users_followers = ReadCSV(colab_users_followers_url, False)
print(colab_users_followers.shape)
colab_users_followers.head(1)
##
df1=colab_users
df2=colab_users_followers
##
df_all = pd.concat([colab_users.set_index('colab_user_id'), colab_users_followers.set_index('colab_user_id')], axis='columns', keys=['First', 'Second'])
df_final = df_all.swaplevel(axis='columns')[colab_users.columns[1:]]
df_final
#
# concatenando usuários com seus seguidores em uma unica planilha sem duplicados
#
result = pd.concat([df1,df2])
idx = np.unique(result["colab_user_id"], return_index=True)[1]
colab_users_and_followers =result.iloc[idx]
colab_users_and_followers
DownloadDataFrame(colab_users_and_followers, "colab_users_and_followers.csv")

validating colab_users file
Index(['colab_user_id', 'gender', 'birth_date', 'city_id', 'city_name',
       'state_id', 'state_name', 'created_at', 'last_sign_in_at', 'device'],
      dtype='object')
File is clean
(49353, 10)
(19665, 10)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>